In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import pandas as pd
import time
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.snowball import DutchStemmer
import sqlite3
import pandas as pd
from sklearn.cluster import KMeans

In [2]:
db = sqlite3.connect('reports.sqlite')

agenda_items = pd.read_sql_query("SELECT * FROM agenda_items", db)
speaker_items = pd.read_sql_query("SELECT * FROM speaker_turns", db)
speakers = pd.read_sql_query("SELECT * FROM speakers", db)

# controle of het inlezen goed gaat
agenda_items.head()
speaker_items.head()

,id,agenda_item,turn_no,speaker,txt
0,1,1,1,1,Detijdelijke voorzitter:Ik deel aan de Kamer m...
1,2,5,1,2,Voorzitter. Ik zal het idee van de heer Pechto...
2,3,5,2,2,Zo mogelijk in aanwezigheid van de minister-pr...
3,4,5,3,3,Ik kan mij die behoefte heel goed voorstellen....
4,5,5,4,4,Steun namens D66 voor het verzoek van de heer ...


In [3]:
speakers.loc[speakers['id'] == 48,'political'] = 'VVD' # minister Rutte behoort tot de VVD
speakers.loc[speakers['id'] == 49,'political'] = 'CDA'# meneer van Dam behoort tot het CDA
speakers.loc[speakers['id'] == 51,'political'] = 'VVD' # minister Hennis-Plasschaert behoort bij de VVD
speakers.loc[speakers['id'] == 54,'political'] = 'PvdA' # staatssecretaris Dijksma behoort tot PvdA
speakers.loc[speakers['id'] == 56,'political'] = 'VVD' # staatssecretaris Wiebes behoort tot VVD
speakers.loc[speakers['id'] == 84,'political'] = 'VVD' # mevrouw Schippers behoort tot de VVD
speakers.loc[speakers['id'] == 94,'political'] = 'PvdA'# Staatssecretaris van Rijn behoort tot de PvdA
speakers.loc[speakers['id'] == 98,'political'] = 'PvdA'# minister Plasterk behoort tot de PvdA
speakers.loc[speakers['id'] == 106,'political'] = 'VVD' # minister Blok behoort tot de VVD
speakers.loc[speakers['id'] == 108,'political'] = 'PvdA' # minister Koenders behoort tot de PvdA
speakers.loc[speakers['id'] == 115,'political'] = 'PvdA' # minister Ploumen behoort tot de PvdA
speakers.loc[speakers['id'] == 118,'political'] = 'VVD' # minister Schultz van Haegen-Maas Geesteranus behoort tot de VVD
speakers.loc[speakers['id'] == 125,'political'] = 'PvdA' # minister Asscher behoort tot de PvdA
speakers.loc[speakers['id'] == 134,'political'] = 'PvdA' # straatssecretaris Kleijnsma behoort tot de PvdA
speakers.loc[speakers['id'] == 136,'political'] = 'PvdA' # minister Ploumen behoort tot de PvdA
speakers.loc[speakers['id'] == 140,'political'] = 'PvdA' # minister Dijsselbloem behoort tot de PvdA
speakers.loc[speakers['id'] == 141,'political'] = 'VVD' # staatssecretaris Dijkhoff behoort tot de VVD
speakers.loc[speakers['id'] == 148,'political'] = 'VVD' # minister Kamp behoort tot de VVD
speakers.loc[speakers['id'] == 149,'political'] = 'VVD' # staatssecretaris Dekker behoort tot de VVD
speakers.loc[speakers['id'] == 157,'political'] = 'PvdA' # minister Bussemaker behoort tot de PvdA
speakers.loc[speakers['id'] == 169,'political'] = 'SP' # mevrouw Leijten behoort tot de SP
speakers.loc[speakers['id'] == 180,'political'] = 'CU' # mevrouw Schouten behoort tot de CU
speakers.loc[speakers['id'] == 181,'political'] = 'VVD' # minister Bruins behoort tot de VVD
speakers.loc[speakers['id'] == 182,'political'] = 'CDA' # minister De Jonge behoort tot de CDA
speakers.loc[speakers['id'] == 183,'political'] = 'CDA' # minister Grapperhaus behoort tot de CDA
speakers.loc[speakers['id'] == 184,'political'] = 'D66' # minister Koolmees behoort tot de D66
speakers.loc[speakers['id'] == 185,'political'] = 'CDA' # minister Bijleveld behoort tot de CDA
speakers.loc[speakers['id'] == 186,'political'] = 'VVD' # minister Dekker behoort tot de VVD
speakers.loc[speakers['id'] == 189,'political'] = 'D66' # minister Kaag behoort tot de D66
speakers.loc[speakers['id'] == 190,'political'] = 'VVD' # minister Knops behoort tot de CDA
speakers.loc[speakers['id'] == 191,'political'] = 'VVD' # minister Zijlstra behoort tot de VVD
speakers.loc[speakers['id'] == 192,'political'] = 'CDA' # minister Hoekstra behoort tot de CDA
speakers.loc[speakers['id'] == 193,'political'] = 'D66' # Staatssecretaris Snel behoort tot de D66
speakers.loc[speakers['id'] == 194,'political'] = 'CU' # minister Slob behoort tot de CU
speakers.loc[speakers['id'] == 195,'political'] = 'VVD' # minister Wiebes behoort tot de VVD
speakers.loc[speakers['id'] == 197,'political'] = 'CDA' # staatssecretaris Keijzer behoort tot de CDA
speakers.loc[speakers['id'] == 198,'political'] = 'D66' # minister Ollogren behoort tot de D66
speakers.loc[speakers['id'] == 203,'political'] = 'VVD' # Staatssecretaris Visser behoort tot de VVD
speakers.loc[speakers['id'] == 204,'political'] = 'VVD' # Staatssecretaris Van Ark behoort tot de VVD
speakers.loc[speakers['id'] == 207,'political'] = 'D66' # Minister Van Engelshoven behoort tot de D66
speakers.loc[speakers['id'] == 210,'political'] = 'VVD' # Minister Van Nieuwenhuizen-Wijbenga behoort tot de VVD
speakers.loc[speakers['id'] == 211,'political'] = 'D66' # Staatssecretaris Van Veldhoven behoort tot de D66
speakers.loc[speakers['id'] == 212,'political'] = 'VVD' # Staatssecretaris Harbers behoort tot de VVD
speakers.loc[speakers['id'] == 213,'political'] = 'CU' # Staatssecretaris Blokhuis behoort tot de CU
speakers.loc[speakers['id'] == 217,'political'] = 'PvdD' # De heer Wassenberg behoort tot de PvdD
speakers.loc[speakers['id'] == 218,'political'] = 'VVD' # De heer Van Meenen behoort tot de D66
speakers.loc[speakers['id'] == 233,'political'] = 'CDA' # Minister Bijleveld-Schouten behoort tot de CDA
speakers.loc[speakers['id'] == 234,'political'] = 'D66' # Staatssecretaris Van Veldhoven-van der Meer behoort tot de D66

In [70]:
speakers = speakers[~speakers['political'].isin(['EP/CDA', 'EP/CU/SGP', 'EP/D66', 
                                           'EP/GroenLinks', 'EP/PvdA', 'EP/PvdD',
                                           'EP/SP', 'EP/VVD'])]
speakers = speakers.dropna()
speakers = speakers.replace('Forum voor Democratie','FvD')
speakers = speakers.replace('ChristenUnie','CU')
print(speakers.groupby('political').count())

            id  prefix  name
political                   
50PLUS       5       5     5
CDA         28      28    28
CU           9       9     9
D66         30      30    30
DENK         3       3     3
FvD          4       4     4
GroenLinks  17      17    17
PVV         21      21    21
PvdA        21      21    21
PvdD         9       9     9
SGP          4       4     4
SP          17      17    17
VVD         59      59    59


Hieronder hebben we de sprekerdataset en wat ze gezegd hebben samengevoegd. Alle tekst van personen die Iedereen die niet tot een politieke partij behoord is uit de datset gehaald. 

In [71]:
merged = speaker_items.merge(speakers, left_on='speaker', right_on='id')[['id_y','txt','political']]

In [84]:
# controle welke partijen er zijn met hoeveel tekst
merged.groupby('political').count().sort_values('id_y')

,id_y,txt
political,,
FvD,2095,2095
SGP,2720,2720
50PLUS,3477,3477
DENK,3773,3773
PvdD,4050,4050
CU,5431,5431
PVV,8442,8442
PvdA,8535,8535
GroenLinks,10093,10093


Uit bovenstaande lijst blijkt dat er door mensen van het europees parlement maar weinig wordt gezegd. Omdat hier zo weinig tekst van beschikbaar is, willen we deze niet meenemen in ons model, en hebben we alle teksten Europese parlementariers uit de dataset verwijderd.

In [18]:
# stemmer
stemmer = DutchStemmer()
analyzer = CountVectorizer().build_analyzer()

def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

Hier maken we een dataframe met elke spreker, hoe vaak hij spreekt en hoe lang hij gemiddeld aan het woord is.

In [85]:
merged['length'] = merged['txt'].apply(len)

In [88]:
kmSpeakers = pd.DataFrame()
kmSpeakers['nTurns'] = merged.groupby('id_y').count()['txt']
kmSpeakers['mTurnLength'] = merged.groupby('id_y').sum()['length'] / kmSpeakers['nTurns']
#kmSpeakers['political'] = merged.groupby('id_y')['political'][1]
kmSpeakers = kmSpeakers.merge(speakers,left_on = 'id_y',right_on = 'id')[['id','nTurns','mTurnLength','political','prefix']]
kmSpeakers

,id,nTurns,mTurnLength,political,prefix
0,2,1345,734.103346,CDA,De heer
1,3,1128,509.812057,SP,Mevrouw
2,4,928,726.934267,D66,De heer
3,5,731,578.785226,PvdA,Mevrouw
4,6,875,681.662857,GroenLinks,De heer
5,7,64,625.921875,FvD,De heer
6,8,256,590.394531,VVD,De heer
7,9,182,769.274725,PVV,Mevrouw
8,10,1701,825.603175,50PLUS,De heer
9,11,1092,720.996337,SGP,De heer


In [94]:
n = merged['political'].nunique()
kmSpeakers['predicted'] = KMeans(n_clusters=n, random_state=0).fit_predict(kmSpeakers[['nTurns','mTurnLength']])
kmSpeakers.head()

,id,nTurns,mTurnLength,political,prefix,predicted
0,2,1345,734.103346,CDA,De heer,12
1,3,1128,509.812057,SP,Mevrouw,2
2,4,928,726.934267,D66,De heer,2
3,5,731,578.785226,PvdA,Mevrouw,11
4,6,875,681.662857,GroenLinks,De heer,2


In [95]:
dfClusters = pd.DataFrame()
for i in range(n):
    dfClusters['cluster' + str(i)] = kmSpeakers[kmSpeakers['predicted'] == i].groupby('political').count()['predicted']
dfClusters

,cluster0,cluster1,cluster2,cluster3,cluster4,cluster5,cluster6,cluster7,cluster8,cluster9,cluster10,cluster11,cluster12
political,,,,,,,,,,,,,
CDA,5,NaN,NaN,NaN,1.0,NaN,NaN,2.0,7,2.0,3,6,2.0
D66,2,4.0,2.0,NaN,NaN,NaN,NaN,1.0,7,3.0,6,5,NaN
PVV,3,1.0,2.0,NaN,NaN,NaN,NaN,NaN,5,NaN,5,5,NaN
PvdA,3,NaN,2.0,1.0,7.0,NaN,1.0,1.0,2,NaN,1,3,NaN
PvdD,1,NaN,2.0,NaN,NaN,NaN,NaN,1.0,1,NaN,3,1,NaN
VVD,12,3.0,NaN,NaN,7.0,NaN,1.0,3.0,7,2.0,17,5,2.0


In [39]:
dfFunctions = pd.DataFrame()
for i in range(n):
    dfFunctions['cluster' + str(i)] = kmSpeakers[kmSpeakers['predicted'] == i].groupby('prefix').count()['predicted']
dfFunctions

,cluster0,cluster1,cluster2,cluster3,cluster4,cluster5,cluster6,cluster7,cluster8,cluster9,cluster10,cluster11,cluster12
prefix,,,,,,,,,,,,,
De heer,23,3.0,1.0,25.0,1,15.0,1.0,3.0,30,9,7.0,2.0,NaN
Mevrouw,20,1.0,NaN,10.0,1,7.0,NaN,1.0,14,8,2.0,NaN,NaN
Staatssecretaris,1,NaN,NaN,NaN,1,NaN,NaN,NaN,1,3,NaN,3.0,6.0


In [12]:
clustered = speakers[speakers['id'] != 1][['id', 'name','political']]
clustered['predicted'] = yhat
clustered.head()

ValueError: Length of values does not match length of index

In [13]:
clusters = pd.DataFrame()
for i in range(n):
    df = clustered[clustered['predicted'] == i]
    clusters['group' + str(i)] = df.groupby('political').count()['predicted']

KeyError: 'predicted'

In [ ]:
clusters[:]